In [ ]:
# # -------------------------------
# # 0. Parameters
# # -------------------------------
# N = 10000  # number of paths
# T = 3
# mu, rho, sigma = 10, 0.0, 2  # i.i.d. prices
# inventory_levels = [0, 1]  # toy storage
# actions = {
#     0: [0, +1],  # I=0 can hold or inject
#     1: [0, -1],  # I=1 can hold or withdraw
# }

# # -------------------------------
# # 1. Simulate price paths
# # -------------------------------
# S1 = mu + sigma * np.random.randn(N)
# S2 = mu + sigma * np.random.randn(N)
# S3 = mu + sigma * np.random.randn(N)

# # Store prices in a matrix for easier indexing
# S = np.column_stack([S1, S2, S3])

# # -------------------------------
# # 2. Terminal value
# # -------------------------------
# # For each inventory state
# V = {}
# for I in inventory_levels:
#     V[(T, I)] = I * S[:, T-1]  # terminal payoff

# # -------------------------------
# # 3. Backward induction: t=T-1 to 1
# # -------------------------------
# for t in reversed(range(1, T)):
#     V_new = {}
#     for I in inventory_levels:
#         # For each path, compute value for each possible action
#         Q_all_actions = np.zeros((N, len(actions[I])))
#         for idx, a in enumerate(actions[I]):
#             # check bounds (I+a in allowed inventory)
#             if 0 <= I + a <= 1:
#                 # cashflow today: inject= -S, withdraw=+S, hold=0
#                 CF = np.where(a==+1, -S[:, t-1], 0) + np.where(a==-1, S[:, t-1], 0)
#                 # continuation value from next step
#                 V_next = V[(t+1, I+a)]
#                 Q_all_actions[:, idx] = CF + V_next
#             else:
#                 Q_all_actions[:, idx] = -np.inf  # illegal action
#         # choose optimal action pathwise
#         V_new[I] = np.max(Q_all_actions, axis=1)
#     V.update({(t, I): V_new[I] for I in inventory_levels})

# # -------------------------------
# # 4. Storage value at t=1, I=0
# # -------------------------------
# V1_0 = V[(1, 0)]
# storage_value = np.mean(V1_0)
# print("Storage value at t=1, I=0:", storage_value)

# # -------------------------------
# # 5. Reconstruct optimal policy for plot
# # -------------------------------
# optimal_actions = {}
# for t in range(1, T):
#     actions_taken = np.zeros(N)
#     for I in inventory_levels:
#         Q_all_actions = np.zeros((N, len(actions[I])))
#         for idx, a in enumerate(actions[I]):
#             if 0 <= I + a <= 1:
#                 CF = np.where(a==+1, -S[:, t-1], 0) + np.where(a==-1, S[:, t-1], 0)
#                 V_next = V[(t+1, I+a)]
#                 Q_all_actions[:, idx] = CF + V_next
#             else:
#                 Q_all_actions[:, idx] = -np.inf
#         # best action pathwise
#         best_idx = np.argmax(Q_all_actions, axis=1)
#         actions_taken[:] = np.array([actions[I][i] for i in best_idx])
#     optimal_actions[t] = actions_taken

# # -------------------------------
# # 6. Plot one path of inventory and actions
# # -------------------------------
# path = 100  # just pick first path
# inventory_path = [0]
# for t in range(1, T):
#     a = optimal_actions[t][path]
#     inventory_path.append(inventory_path[-1]+a)

# plt.figure(figsize=(8,4))
# plt.step(range(T), inventory_path, where='post', label='Inventory')
# plt.scatter(range(1, T), [optimal_actions[t][path] for t in range(1, T)], 
#             color='red', label='Action (inject=+1, withdraw=-1)')
# plt.xticks(range(T))
# plt.xlabel('Time step')
# plt.ylabel('Inventory / Action')
# plt.title('Toy Storage: Optimal Inventory and Actions (path 0)')
# plt.grid(True)
# plt.legend()
# plt.show()

In [ ]:
# I_max = 100.0
# q_in = 5.0
# q_out = 5.0

# n_I = 51
# I_grid = np.linspace(0.0, I_max, n_I)


# n_paths, n_steps = paths.shape

# V = np.zeros((n_paths, n_I))
# S_T = paths[:, -1]

# for i, I in enumerate(I_grid):
#     V[:, i] = I * S_T

# actions = np.array([-q_out, 0.0, q_in])

# def continuation_value(S, Y):
#     # regress Y on [1, S, S^2]
#     X = np.column_stack([np.ones_like(S), S, S**2])
#     beta, _, _, _ = np.linalg.lstsq(X, Y, rcond=None)
#     return X @ beta

# for t in reversed(range(n_steps - 1)):
#     print(t)
#     S_t = paths[:, t]
#     V_next = V.copy()

#     for i, I in enumerate(I_grid):
#         values = []

#         for a in actions:
#             I_new = I + a
#             if I_new < 0 or I_new > I_max:
#                 values.append(np.full_like(S_t, -np.inf))
#                 continue

#             j = np.argmin(np.abs(I_grid - I_new))

#             cont = continuation_value(S_t, V_next[:, j])
#             payoff = -a * S_t + cont
#             values.append(payoff)

#         V[:, i] = np.max(np.column_stack(values), axis=1)


# I0 = 50.0
# i0 = np.argmin(np.abs(I_grid - I0))

# storage_value = V[:, i0].mean()
# print(f"Estimated storage value: {storage_value:.2f}")